<a href="https://colab.research.google.com/github/GratsiaMariia/python-crm-data-analysis-x-school/blob/main/Final_Project_task_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
calls = pd.read_excel('Calls (Done).xlsx')

In [ ]:
contacts = pd.read_excel('Contacts (Done).xlsx')

In [ ]:
spend = pd.read_excel('Spend (Done).xlsx')

In [ ]:
deals = pd.read_excel('Deals (Done).xlsx')

## Remove duplicate and irrelevant columns.
## Process missing values accordingly
## Convert data types for columns such as dates and numeric values.

In [ ]:
databases = {
    'Calls': calls,
    'Contacts': contacts,
    'Deals': deals,
    'Spend': spend
}

\Remove duplicates and see the report for each table

In [ ]:
for name, data in databases.items():
    original_shape = data.shape[0] # data.shape returns the number of rows and columns - [0]- number of rows
    data.drop_duplicates(inplace=True)
    data.reset_index(drop=True, inplace=True)
    num_duplicates_removed = original_shape - data.shape[0]
    print(f'{name}:')
    print(f'Duplicates removed: {num_duplicates_removed}')
    print(data.info())
    print()

Calls:
Duplicates removed: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Dialled Number              0 non-null      float64
 8   Outgoing Call Status        86875 non-null  object 
 9   Scheduled in CRM            86875 non-null  float64
 10  Tag                         0 non-null      float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.0+ MB
None

Contacts:
Duplicates removed: 0
<class 'pandas.core.f

# **CALLS**

On the Calls:
1. delete empty columns


In [ ]:
calls = calls.drop(columns=['Dialled Number', 'Tag'])
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95791 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Outgoing Call Status        86875 non-null  object 
 8   Scheduled in CRM            86875 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 6.6+ MB


2. we check the Call Duration column (in seconds) by connections with Status and Outgoing status - the result is that the cells that are not filled are canceled and overdue - so we fill them with zeros.

In [ ]:
grouped_call_status = calls.groupby(calls['Call Duration (in seconds)'].isna())['Call Status'].value_counts()
print(grouped_call_status)

Call Duration (in seconds)  Call Status               
False                       Attended Dialled              70703
                            Unattended Dialled            16030
                            Missed                         5922
                            Received                       3077
                            Scheduled Attended Delay         22
                            Scheduled Unattended Delay       17
                            Scheduled Attended               14
                            Scheduled Unattended              6
True                        Overdue                          60
                            Cancelled                        20
                            Scheduled                         3
Name: count, dtype: int64


In [ ]:
grouped_call_outgoing = calls.groupby(calls['Call Duration (in seconds)'].isna())['Outgoing Call Status'].value_counts()
print(grouped_call_outgoing)

Call Duration (in seconds)  Outgoing Call Status
False                       Completed               86792
True                        Overdue                    60
                            Cancelled                  20
                            Scheduled                   3
Name: count, dtype: int64


In [ ]:
calls['Call Duration (in seconds)'] = calls['Call Duration (in seconds)'].fillna(0)
calls['Call Duration (in seconds)'].isna().sum()

0

3. check the Scheduled in CRM column by links to the Status - the result is that the cells that are not filled are canceled and overdue - so check 'Call Status'. If the call status is Attended Dialled or Unattended Dialled, the function returns False. Otherwise it returns True.

In [ ]:
grouped_data = calls.groupby('Scheduled in CRM')['Call Status'].value_counts()
print(grouped_data)

Scheduled in CRM  Call Status               
0.0               Attended Dialled              70703
                  Unattended Dialled            16030
1.0               Overdue                          60
                  Scheduled Attended Delay         22
                  Cancelled                        20
                  Scheduled Unattended Delay       17
                  Scheduled Attended               14
                  Scheduled Unattended              6
                  Scheduled                         3
Name: count, dtype: int64


In [ ]:
def fill_scheduled_in_crm(row):
    if pd.isna(row['Scheduled in CRM']):
        if row['Call Status'] in ['Attended Dialled', 'Unattended Dialled']:
            return False
        else:
            return True
    else:
        return row['Scheduled in CRM']

calls['Scheduled in CRM'] = calls.apply(fill_scheduled_in_crm, axis=1)

calls['Scheduled in CRM'].isna().sum()

0

4. check the Outgoing Call Status column by links to the Status - the result - uncompleted cells are Missed', 'Received' - fill them with Uncompleted.

In [ ]:
grouped_data_outgoing_call = calls.groupby(calls['Outgoing Call Status'].isna())['Call Status'].value_counts()
print(grouped_data_outgoing_call)

Outgoing Call Status  Call Status               
False                 Attended Dialled              70703
                      Unattended Dialled            16030
                      Overdue                          60
                      Scheduled Attended Delay         22
                      Cancelled                        20
                      Scheduled Unattended Delay       17
                      Scheduled Attended               14
                      Scheduled Unattended              6
                      Scheduled                         3
True                  Missed                         5922
                      Received                       3077
Name: count, dtype: int64


In [ ]:
def fill_outgoing_call_status(row):
    if pd.isna(row['Outgoing Call Status']):
        if row['Call Status'] in ['Missed', 'Received']:
            return 'Uncompleted'
        else:
            return True
    else:
        return row['Outgoing Call Status']

calls['Outgoing Call Status'] = calls.apply(fill_outgoing_call_status, axis=1)

calls['Outgoing Call Status'].isna().sum()

0

In [ ]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Id                          95874 non-null  int64  
 1   Call Start Time             95874 non-null  object 
 2   Call Owner Name             95874 non-null  object 
 3   CONTACTID                   91941 non-null  float64
 4   Call Type                   95874 non-null  object 
 5   Call Duration (in seconds)  95874 non-null  float64
 6   Call Status                 95874 non-null  object 
 7   Outgoing Call Status        95874 non-null  object 
 8   Scheduled in CRM            95874 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 6.6+ MB


**Change the data type according to the values**

In [ ]:
calls['Call Start Time'] = pd.to_datetime(calls['Call Start Time'], format='%d.%m.%Y %H:%M')

calls['Call Duration (in seconds)'] = calls['Call Duration (in seconds)'].astype(int)

calls['Scheduled in CRM'] = calls['Scheduled in CRM'].astype('bool')

**Process Contact Id**

We try to replace the missing values in the CONTACTID column with ID values from the CONTACTS table.

To do this, join the calls and contacts tables, taking into account Names and Dates of creation
Do the same with the deals table.
The remaining values are filled with [1000 + i for i in range(1, len(null_indices) + 1)] so that they are not zeros (just in case).

In [ ]:
calls['CONTACTID'].isna().sum()

3933

In [ ]:
contacts['Created Time'] = pd.to_datetime(contacts['Created Time'], format='%d.%m.%Y %H:%M')

In [ ]:
merged_df = pd.merge(calls, contacts,
                     how='inner',
                     left_on=['Call Owner Name', 'Call Start Time'],
                     right_on=['Contact Owner Name', 'Created Time'])

merged_df.columns


Index(['Id_x', 'Call Start Time', 'Call Owner Name', 'CONTACTID', 'Call Type',
       'Call Duration (in seconds)', 'Call Status', 'Outgoing Call Status',
       'Scheduled in CRM', 'Id_y', 'Contact Owner Name', 'Created Time',
       'Modified Time'],
      dtype='object')

In [ ]:
calls.loc[calls['CONTACTID'].isna(), 'CONTACTID'] = merged_df['Id_y']

calls['CONTACTID'].isna().sum()

3913

In [ ]:
deals["Created Time"] = pd.to_datetime(deals["Created Time"], format="%d.%m.%Y %H:%M")

In [ ]:
merged_df_deals = pd.merge(calls, deals,
                     how='inner',
                     left_on=['Call Owner Name', 'Call Start Time'],
                     right_on=['Deal Owner Name', 'Created Time'])

merged_df.columns

Index(['Id_x', 'Call Start Time', 'Call Owner Name', 'CONTACTID', 'Call Type',
       'Call Duration (in seconds)', 'Call Status', 'Outgoing Call Status',
       'Scheduled in CRM', 'Id_y', 'Contact Owner Name', 'Created Time',
       'Modified Time'],
      dtype='object')

In [ ]:
calls.loc[calls['CONTACTID'].isna(), 'CONTACTID'] = merged_df_deals['Id_y']

calls['CONTACTID'].isna().sum()

3910

In [ ]:
null_indices = calls[calls['CONTACTID'].isna()].index

calls.loc[null_indices, 'CONTACTID'] = [1000 + i for i in range(1, len(null_indices) + 1)]

calls['CONTACTID'].isna().sum()

0

In [ ]:
calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          95874 non-null  int64         
 1   Call Start Time             95874 non-null  datetime64[ns]
 2   Call Owner Name             95874 non-null  object        
 3   CONTACTID                   95874 non-null  float64       
 4   Call Type                   95874 non-null  object        
 5   Call Duration (in seconds)  95874 non-null  int64         
 6   Call Status                 95874 non-null  object        
 7   Outgoing Call Status        95874 non-null  object        
 8   Scheduled in CRM            95874 non-null  bool          
dtypes: bool(1), datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 5.9+ MB


# **CONTACTS**

Since the data in this table has been filled in, we only change the data type Created Time (the column was converted when processing Calls)

In [ ]:
contacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18548 non-null  int64         
 1   Contact Owner Name  18548 non-null  object        
 2   Created Time        18548 non-null  datetime64[ns]
 3   Modified Time       18548 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 579.8+ KB


In [ ]:
# contacts['Created Time'] = pd.to_datetime(contacts['Created Time'], format='%d.%m.%Y %H:%M')
contacts['Modified Time'] = pd.to_datetime(contacts['Modified Time'], format='%d.%m.%Y %H:%M')

In [ ]:
contacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18548 entries, 0 to 18547
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Id                  18548 non-null  int64         
 1   Contact Owner Name  18548 non-null  object        
 2   Created Time        18548 non-null  datetime64[ns]
 3   Modified Time       18548 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(1)
memory usage: 579.8+ KB


# **SPEND**

Split Campaign and Fill Campaign, AdGroup, Ad - Unknown

In [ ]:
spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19862 entries, 0 to 19861
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         19862 non-null  datetime64[ns]
 1   Source       19862 non-null  object        
 2   Campaign     14785 non-null  object        
 3   Impressions  19862 non-null  int64         
 4   Spend        19862 non-null  float64       
 5   Clicks       19862 non-null  int64         
 6   AdGroup      13951 non-null  object        
 7   Ad           13951 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 1.2+ MB


**separation spend['Campaign']**

Split the values in the column into name and date. Separate the date into a separate column

In [ ]:
spend['Campaign'].nunique()

51

In [ ]:
spend['Campaign Date'] = spend['Campaign'].str.extract(r'(\d{2}\.\d{2}\.\d{2,4})', expand=False)
spend['Campaign Date'] = pd.to_datetime(spend['Campaign Date'], format='%d.%m.%y', errors='coerce')
spend['Campaign'] = spend['Campaign'].str.replace(r'\d{2}\.\d{2}\.\d{2,4}', '', regex=True)


In [ ]:
spend['Campaign'].unique()

array(['gen_analyst_DE', 'performancemax_eng_DE', nan, 'women', 'wide_DE',
       'interests_DE', 'recentlymoved_DE', 'LAL_DE', 'wide_com_DE',
       'b_DE', 'youtube_shorts_DE', 'comp_search_DE',
       'brand_search_eng_DE', 'web2408_DE', 'interests_Uxui_DE',
       'discovery_DE', 'retargeting_DE', 'wide_gos_DE', 'wide_webinar_DE',
       'blog2_DE', 'test_DE', 'bbo_DE', 'berlin_dd_DE', 'wide_AT',
       'recentlymoved_AT', 'youtube_shortsin_AT', 'discovery_wide1_AT',
       'performancemax_wide_AT', '_widde_PL', 'wide_PL',
       'interests_WebDev_AT', 'interests_WebDev_PL', 'shorts_PL',
       '1performancemax_wide_PL', 'recentlymoved_PL', 'women_PL',
       'LAL_ab__PL'], dtype=object)

There's one campaign left with a date, convert it.


In [ ]:
date = pd.to_datetime('24.08.2023', format='%d.%m.%Y')
spend.loc[spend['Campaign'] == 'web2408_DE', 'Campaign Date'] = date
spend.loc[spend['Campaign'] == 'web2408_DE', 'Campaign'] = 'web_DE'

In [ ]:
spend['Campaign'].unique()

array(['gen_analyst_DE', 'performancemax_eng_DE', nan, 'women', 'wide_DE',
       'interests_DE', 'recentlymoved_DE', 'LAL_DE', 'wide_com_DE',
       'b_DE', 'youtube_shorts_DE', 'comp_search_DE',
       'brand_search_eng_DE', 'web_DE', 'interests_Uxui_DE',
       'discovery_DE', 'retargeting_DE', 'wide_gos_DE', 'wide_webinar_DE',
       'blog2_DE', 'test_DE', 'bbo_DE', 'berlin_dd_DE', 'wide_AT',
       'recentlymoved_AT', 'youtube_shortsin_AT', 'discovery_wide1_AT',
       'performancemax_wide_AT', '_widde_PL', 'wide_PL',
       'interests_WebDev_AT', 'interests_WebDev_PL', 'shorts_PL',
       '1performancemax_wide_PL', 'recentlymoved_PL', 'women_PL',
       'LAL_ab__PL'], dtype=object)

In [ ]:
spend['Campaign'].nunique()

36

In [ ]:
columns_to_fill_spend = ['Campaign',
                          'AdGroup',
                         'Ad'
                        ]

spend.loc[:, columns_to_fill_spend] = spend.loc[:, columns_to_fill_spend].fillna('Unknown')

In [ ]:
spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19862 entries, 0 to 19861
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           19862 non-null  datetime64[ns]
 1   Source         19862 non-null  object        
 2   Campaign       19862 non-null  object        
 3   Impressions    19862 non-null  int64         
 4   Spend          19862 non-null  float64       
 5   Clicks         19862 non-null  int64         
 6   AdGroup        19862 non-null  object        
 7   Ad             19862 non-null  object        
 8   Campaign Date  9815 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 1.4+ MB


# **DEALS**

In [ ]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21592 entries, 0 to 21591
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Id                   21590 non-null  float64       
 1   Deal Owner Name      21561 non-null  object        
 2   Closing Date         14643 non-null  object        
 3   Quality              19338 non-null  object        
 4   Stage                21590 non-null  object        
 5   Lost Reason          16122 non-null  object        
 6   Page                 21590 non-null  object        
 7   Campaign             16066 non-null  object        
 8   SLA                  15533 non-null  object        
 9   Content              14146 non-null  object        
 10  Term                 12453 non-null  object        
 11  Source               21590 non-null  object        
 12  Payment Type         496 non-null    object        
 13  Product              3592 non-n

According to the data 2 values were not filled in Id.
After checking these values it was found out that the rows with unfilled Id are empty, so delete these rows

In [ ]:
deals[deals['Id'].isna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,...,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch
21590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,#REF!,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
deals.dropna(subset=['Id'], inplace=True)
print(deals['Id'].isna().sum())

0


**separation deals['Campaign']**

Split the values in the column into name and date. Separate the date into a separate column

In [ ]:
deals['Campaign'].nunique()

154

In [ ]:
deals['Campaign Date'] = deals['Campaign'].str.extract(r'(\d{2}\.\d{2}\.\d{2,4})', expand=False)
deals['Campaign Date'] = pd.to_datetime(deals['Campaign Date'], format='%d.%m.%y', errors='coerce')
deals['Campaign'] = deals['Campaign'].str.replace(r'\d{2}\.\d{2}\.\d{2,4}', '', regex=True)


In [ ]:
deals['Campaign'].unique()

array(['women', nan, 'engwien_AT', 'recentlymoved_DE', 'discovery_DE',
       'youtube_shorts_DE', 'brand_search_eng_DE', '1406start',
       'interests_DE', 'performancemax_eng_DE', 'wide_DE', '1006start',
       'retargeting_DE', 'germany_DE', 'performancemax_wide_AT', 'LAL_DE',
       'webinar1906', 'germania_DE', 'blog_DE', 'blog2_DE', 'wide_AT',
       'Jobs_germany_DE', '2005_Lost_DE', 'uk_DE', 'wide_webinar_DE',
       'Akademia', 'BloggerIvan', 'Genie_DE', 'Live_DE', '1706_DE',
       'performancemax_digitalmarkt_ru_DE', 'interests_Uxui_DE',
       '5555_DE', 'ASA_de_DE', '2905start', 'webinar1604',
       'bloggerfrai_DE', 'bloggerdr_DE', 'Trigger_DE', 'Bloggerel_DE',
       'Forum_DE', 'Consult_DE', 'work_DE', 'Bolgspeak_DE', 'tyk_DE',
       'of_DE', 'Berlin_DE', 'Markt_DE', 'BloggerShina_DE',
       'berlin_dd_DE', 'Relocat_DE', 'col_DE', 'Blogff_DE', 'eha_DE',
       'jet_DE', 'test_DE', '1005start', 'workingin_DE', 'bbo_DE',
       'Kak_PL', 'price_DE', 'hb_DE', '1004star

Since the remaining values are dd.mm, we will extract these dates as well. Given that the range of dates on this data - April 2023 - June 2024, we can assume that the campaigns that were in the period January-March, it is 2024, the rest of 2023 (allow the error April-June)

In [ ]:
deals['Campaign Date 2'] = deals['Campaign'].str.extract(r'(\d{4}|\d{2}\.\d{2})', expand=False)

def correct_year(date):
    if isinstance(date, str):
        if len(date) == 4:
            day, month = date[:2], date[2:]
            if int(month) < 4:
                return f"{day}.{month}.24"
            else:
                return f"{day}.{month}.23"
        elif len(date) == 5:
            day, month = date.split('.')
            if int(month) < 4:
                return f"{day}.{month}.24"
            else:
                return f"{day}.{month}.23"
    return date

deals['Campaign Date 2'] = deals['Campaign Date 2'].astype(str)
deals['Campaign Date 2'] = deals['Campaign Date 2'].apply(correct_year)

deals['Campaign Date 2'] = pd.to_datetime(deals['Campaign Date 2'], format='%d.%m.%y', errors='coerce')

deals['Campaign Date'] = deals['Campaign Date'].combine_first(deals['Campaign Date 2'])

deals = deals.drop('Campaign Date 2', axis=1)

deals['Campaign'] = deals['Campaign'].str.replace(r'\d{1,2}\.\d{1,2}\.\d{4}|\d{4}|\d{1,2}\.\d{1,2}', '', regex=True)

In [ ]:
deals['Campaign'].unique()

array(['women', nan, 'engwien_AT', 'recentlymoved_DE', 'discovery_DE',
       'youtube_shorts_DE', 'brand_search_eng_DE', 'start',
       'interests_DE', 'performancemax_eng_DE', 'wide_DE',
       'retargeting_DE', 'germany_DE', 'performancemax_wide_AT', 'LAL_DE',
       'webinar', 'germania_DE', 'blog_DE', 'blog2_DE', 'wide_AT',
       'Jobs_germany_DE', '_Lost_DE', 'uk_DE', 'wide_webinar_DE',
       'Akademia', 'BloggerIvan', 'Genie_DE', 'Live_DE', '_DE',
       'performancemax_digitalmarkt_ru_DE', 'interests_Uxui_DE',
       'ASA_de_DE', 'bloggerfrai_DE', 'bloggerdr_DE', 'Trigger_DE',
       'Bloggerel_DE', 'Forum_DE', 'Consult_DE', 'work_DE',
       'Bolgspeak_DE', 'tyk_DE', 'of_DE', 'Berlin_DE', 'Markt_DE',
       'BloggerShina_DE', 'berlin_dd_DE', 'Relocat_DE', 'col_DE',
       'Blogff_DE', 'eha_DE', 'jet_DE', 'test_DE', 'workingin_DE',
       'bbo_DE', 'Kak_PL', 'price_DE', 'hb_DE', 'b_DE', 'uk12_AT',
       'ggg10_DE', 'Dis_DE', '1performancemax_wide_PL', 'wide_PL',
       'Blo

In [ ]:
deals['Campaign'].nunique()

129

Replacing missing values in categorical columns - Unknown.

I checked the blank cells for Course duration and basically, if Product is not filled, then Course duration is not filled, but there are 2 exceptions. It could be put by analogy, but these Product is nowhere Course duration. Therefore, we don't fill it in.

In [ ]:
columns_to_fill = ['Deal Owner Name',
                   'Quality',
                   'Lost Reason',
                   'Campaign',
                   'Content',
                   'Term',
                   'Payment Type',
                   'Product',
                   'Education Type',
                   'City',
                   'Level of Deutsch'
                   ]

deals[columns_to_fill] = deals[columns_to_fill].fillna('Unknown')


In [ ]:
deals.loc[deals['City'] == '-', 'City'] = 'Unknown'
deals.loc[deals['City'] == '-', 'City'].sum()

0

**Contact Name**

Contact Name in this table is Id, so we try to fill in the missing values from the tables contacts and calls

The values that are left unfilled are filled in - [2000 + i for i in range(1, len(null_indices_d) + 1)].

In [ ]:
deals['Contact Name'].isna().sum()

61

In [ ]:
merged_deals_id = pd.merge(deals, contacts,
                           how='inner',
                           left_on=['Deal Owner Name', 'Created Time'],
                           right_on=['Contact Owner Name', 'Created Time'])

merged_deals_id.columns

Index(['Id_x', 'Deal Owner Name', 'Closing Date', 'Quality', 'Stage',
       'Lost Reason', 'Page', 'Campaign', 'SLA', 'Content', 'Term', 'Source',
       'Payment Type', 'Product', 'Education Type', 'Created Time',
       'Course duration', 'Months of study', 'Initial Amount Paid',
       'Offer Total Amount', 'Contact Name', 'City', 'Level of Deutsch',
       'Campaign Date', 'Id_y', 'Contact Owner Name', 'Modified Time'],
      dtype='object')

In [ ]:
deals.loc[deals['Contact Name'].isna(), 'Contact Name'] = merged_deals_id['Id_y']

deals['Contact Name'].isna().sum()

32

In [ ]:
merged_deals_calls = pd.merge(deals, calls,
                              how='inner',
                              left_on=['Deal Owner Name', 'Created Time'],
                              right_on=['Call Owner Name', 'Call Start Time'])

merged_deals_calls.columns

Index(['Id_x', 'Deal Owner Name', 'Closing Date', 'Quality', 'Stage',
       'Lost Reason', 'Page', 'Campaign', 'SLA', 'Content', 'Term', 'Source',
       'Payment Type', 'Product', 'Education Type', 'Created Time',
       'Course duration', 'Months of study', 'Initial Amount Paid',
       'Offer Total Amount', 'Contact Name', 'City', 'Level of Deutsch',
       'Campaign Date', 'Id_y', 'Call Start Time', 'Call Owner Name',
       'CONTACTID', 'Call Type', 'Call Duration (in seconds)', 'Call Status',
       'Outgoing Call Status', 'Scheduled in CRM'],
      dtype='object')

In [ ]:
deals.loc[deals['Contact Name'].isna(), 'Contact Name'] = merged_deals_calls['Id_y']

deals['Contact Name'].isna().sum()

32

In [ ]:
null_indices_deals = deals[deals['Contact Name'].isna()].index
null_indices_d = deals[deals['Contact Name'].isna()].index.tolist()

deals.loc[null_indices_deals, 'Contact Name'] = [2000 + i for i in range(1, len(null_indices_d) + 1)]

deals['Contact Name'].isna().sum()

0

Changing the data type.
The data type in Created Time was changed while processing the calls table

In [ ]:
deals['Id'] = deals['Id'].astype(int)
deals['Contact Name'] = deals['Contact Name'].astype(int)
deals['Closing Date'] = pd.to_datetime(deals['Closing Date'], format='%d.%m.%Y')
# deals['Created Time'] = pd.to_datetime(deals['Created Time'], format='%d.%m.%Y %H:%M')

deals['SLA'] = deals['SLA'].astype(str)
deals['SLA'] = pd.to_timedelta(deals['SLA'], errors='coerce')

To handle columns with sums, we remove extra characters and then change the data type

In [ ]:
deals['Initial Amount Paid'] = deals['Initial Amount Paid'].astype(str) \
    .str.replace('€', '', regex=False) \
    .str.replace(' ', '', regex=False) \
    .str.replace('.', '', regex=False) \
    .str.replace(',', '.', regex=False)

deals['Offer Total Amount'] = deals['Offer Total Amount'].astype(str) \
    .str.replace('€', '', regex=False) \
    .str.replace(' ', '', regex=False) \
    .str.replace('.', '', regex=False) \
    .str.replace(',', '.', regex=False)

deals['Initial Amount Paid'] = pd.to_numeric(deals['Initial Amount Paid'], errors='coerce')
deals['Offer Total Amount'] = pd.to_numeric(deals['Offer Total Amount'], errors='coerce')

In [ ]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21590 entries, 0 to 21589
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Id                   21590 non-null  int64          
 1   Deal Owner Name      21590 non-null  object         
 2   Closing Date         14643 non-null  datetime64[ns] 
 3   Quality              21590 non-null  object         
 4   Stage                21590 non-null  object         
 5   Lost Reason          21590 non-null  object         
 6   Page                 21590 non-null  object         
 7   Campaign             21590 non-null  object         
 8   SLA                  15533 non-null  timedelta64[ns]
 9   Content              21590 non-null  object         
 10  Term                 21590 non-null  object         
 11  Source               21590 non-null  object         
 12  Payment Type         21590 non-null  object         
 13  Product              

# **file saving**

In [ ]:
databases = {
    'Calls': calls,
    'Contacts': contacts,
    'Deals': deals,
    'Spend': spend
}

In [ ]:
for name, df in databases.items():
    # File name generation
    file_name = f'new_{name}.xlsx'

    try:
        # Saving DataFrame to Excel file
        df.to_excel(file_name, index=False)
        print(f'Data saved successfully to {file_name}!')
    except Exception as e:
        print(f'Error saving {file_name}: {e}')


Data saved successfully to new_Calls.xlsx!
Data saved successfully to new_Contacts.xlsx!
Data saved successfully to new_Deals.xlsx!
Data saved successfully to new_Spend.xlsx!


In [ ]:
deals.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21590 entries, 0 to 21589
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   Id                   21590 non-null  int64          
 1   Deal Owner Name      21590 non-null  object         
 2   Closing Date         14643 non-null  datetime64[ns] 
 3   Quality              21590 non-null  object         
 4   Stage                21590 non-null  object         
 5   Lost Reason          21590 non-null  object         
 6   Page                 21590 non-null  object         
 7   Campaign             21590 non-null  object         
 8   SLA                  15533 non-null  timedelta64[ns]
 9   Content              21590 non-null  object         
 10  Term                 21590 non-null  object         
 11  Source               21590 non-null  object         
 12  Payment Type         21590 non-null  object         
 13  Product              